In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns
import numpy as np

In [ ]:
df_train = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head()

In [ ]:
plt.hist(df_train['target'], bins=2)

In [ ]:
df_train.drop(columns=['qid'], inplace=True)

In [ ]:
#def
max_feature = 50000
embeming_size = 300
maxlen = 100
from keras.preprocessing.text import Tokenizer
train_x = df_train['question_text'].fillna("_na_").values

train_data = Tokenizer(num_words=max_feature)
train_data.fit_on_texts(list(train_x))
train = train_data.texts_to_sequences(train_x)

In [ ]:
train

In [ ]:
from keras.preprocessing.sequence import pad_sequences
train = pad_sequences(train, maxlen=maxlen)

In [ ]:
train.shape

In [ ]:
labels = df_train['target'].values


In [ ]:
x_train ,x_val, y_train, y_val = train_test_split(train, labels, test_size=0.1, shuffle=True)

In [ ]:
import torch
from torch import nn,optim
from torch.utils.data import DataLoader

In [ ]:
class Dataset():
    def __init__(self):
        self.x = x_train
        self.y = y_train
        self.len = x_train.shape[0]
    def __getitem__(self,index):
        return self.x[index], self.y[index]
    def __len__(self):
        return x_train.shape[0]
    
    

In [ ]:
dataset = DataLoader(Dataset(), batch_size=32)

In [ ]:
import torch.nn.functional as F
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.embed = nn.Embedding(max_feature, embeming_size)
        self.bidirect = nn.LSTM(embeming_size,100, 1, bidirectional=True)
        self.drop = nn.Dropout(0.4)
        self.glob = nn.AvgPool1d(3)
        self.liner1 = nn.Linear(6600,100)
        self.liner2 = nn.Linear(100,1)
        self.flat = nn.Flatten()

    def forward(self, x):
      
        x = self.embed(x)
        x = self.bidirect(x)
        x = self.glob(x[0])
        x = self.drop(x)
        x = self.flat(x)
        x = F.relu(self.liner1(x))
        x = F.sigmoid(self.liner2(x))
        return x

In [ ]:
from torch.optim import Adam
model = model().cuda() #move model to cuda
optimer = AdamW(params=model.parameters(), lr = 0.01)

In [ ]:
loss_f= nn.BCELoss()

In [ ]:
plt.hist(y_val, bins=2)

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')
print(use_cuda)

In [ ]:
from tqdm import tqdm_notebook
from torch.autograd import Variable
loss_value = []
def train_model(epochs):
    
    for i in range(epochs):
        loss_e = 0
        for x,y in tqdm_notebook(dataset):
            x = Variable(x).cuda(device) #move tensor to cuda
            y = Variable(y).cuda(device) #move 130/3 faster
            optimer.zero_grad()
            ypred = model(x.long())
            
            loss = loss_f(ypred, y.float())
            loss.backward()
            optimer.step()
            loss_e  = loss_e + loss
        loss_value.append(loss_e)
    return loss_value
        

In [ ]:
train = train_model(2)

In [ ]:
train

In [ ]:
class Dataset_val():
    def __init__(self):
        self.x = x_val
        self.y = y_val
        self.len = x_val.shape[0]
    def __getitem__(self,index):
        return self.x[index], self.y[index]
    def __len__(self):
        return x_val.shape[0]
data_val = DataLoader(Dataset_val(), batch_size=32)  

In [ ]:
def evalution():
    corr = 0
    all = 0
    with torch.no_grad():
        for x,y in tqdm_notebook(data_val):
                x = Variable(x).cuda(device) #move tensor to cuda
                y = Variable(y).cuda(device) #move 130/3 faster
                
                ypred = model(x.long())
                print(ypred,y)
                
                all = all +x.shape[0]
                if all ==100:
                    break
                #corr = corr + (np.argmax(ypred, axis=1) == y).sum()
    return corr/all


In [ ]:
evalution()

In [ ]:
import math
import torch
from torch.optim.optimizer import Optimizer

class AdamW(Optimizer):
    """Implements Adam algorithm.
    It has been proposed in `Adam: A Method for Stochastic Optimization`_.
    Arguments:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 1e-3)
        betas (Tuple[float, float], optional): coefficients used for computing
            running averages of gradient and its square (default: (0.9, 0.999))
        eps (float, optional): term added to the denominator to improve
            numerical stability (default: 1e-8)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        amsgrad (boolean, optional): whether to use the AMSGrad variant of this
            algorithm from the paper `On the Convergence of Adam and Beyond`_
    .. _Adam\: A Method for Stochastic Optimization:
        https://arxiv.org/abs/1412.6980
    .. _On the Convergence of Adam and Beyond:
        https://openreview.net/forum?id=ryQu7f-RZ
    """

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0, amsgrad=False):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay, amsgrad=amsgrad)
        super(AdamW, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AdamW, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('amsgrad', False)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('Adam does not support sparse gradients, please consider SparseAdam instead')
                amsgrad = group['amsgrad']

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p.data)
                    if amsgrad:
                        # Maintains max of all exp. moving avg. of sq. grad. values
                        state['max_exp_avg_sq'] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                if amsgrad:
                    max_exp_avg_sq = state['max_exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                # if group['weight_decay'] != 0:
                #     grad = grad.add(group['weight_decay'], p.data)

                # Decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                if amsgrad:
                    # Maintains the maximum of all 2nd moment running avg. till now
                    torch.max(max_exp_avg_sq, exp_avg_sq, out=max_exp_avg_sq)
                    # Use the max. for normalizing running avg. of gradient
                    denom = max_exp_avg_sq.sqrt().add_(group['eps'])
                else:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])

                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                step_size = group['lr'] * math.sqrt(bias_correction2) / bias_correction1

                # p.data.addcdiv_(-step_size, exp_avg, denom)
                p.data.add_(-step_size,  torch.mul(p.data, group['weight_decay']).addcdiv_(1, exp_avg, denom) )

        return loss
LR = 0.01
WD = 0.1
optim_W  = AdamW(model.parameters(), lr=LR, betas=(0.9, 0.99), weight_decay = WD)
